In [2]:
# Imports and data reading from files
import pandas as pd
from collections import Counter

data = pd.read_csv('../../data/games/2021-22.csv')
players = pd.read_csv('../../data/players/2021-22.csv')
teams = pd.read_csv('../../data/teams.csv')

# Splitting raw teams data into east and west conferences
conferences = { 'east' : list(teams[teams['location'] == 'east']['team']), 'west' : list(teams[teams['location'] == 'west' ]['team']) }

In [3]:
winning_teams = []
losing_teams = []

# Looping through all the games
for idx, game in data.iterrows():
    
    # Getting team names from data
    team1 = game['Visitor/Neutral']
    team2 = game['Home/Neutral']
    
    # Getting scores from data
    score1 = game['PTS']
    score2 = game['PTS.1']

    # If team1 won, add team1 to winning teams, and team 2 to losing teams
    if score1 > score2:
        winning_teams.append(team1)
        losing_teams.append(team2)
    
    # If team2 won, add team2 to winning teams, and team 1 to losing teams
    if score2 > score1:
        winning_teams.append(team2)
        losing_teams.append(team1)
        
# Initialising key-value data structure for amount of team wins
team_rankings = {"west":{},"east":{}}

# Counting instances of teams in winning and losing teams
wins = Counter(winning_teams)
losses = Counter(losing_teams)

# Populating data structure with win/loss values for Western conference teams
for team in conferences["west"]:
    team_rankings["west"][team] = {}
    team_rankings["west"][team]["Wins"] = wins[team]
    team_rankings["west"][team]["Losses"] = losses[team]
    team_rankings["west"][team]["W-L ratio"] = wins[team] / (wins[team]+losses[team])

# Populating data structure with win/loss values for Eastern conference teams
for team in conferences["east"]:
    team_rankings["east"][team] = {}
    team_rankings["east"][team]["Wins"] = wins[team]
    team_rankings["east"][team]["Losses"] = losses[team]
    team_rankings["east"][team]["W-L ratio"] = wins[team] / (wins[team]+losses[team])

# Creating matrices from data structure
team_rankings_west_df = pd.DataFrame.from_dict(team_rankings["west"], orient='index').sort_values("Wins", ascending=False)
team_rankings_east_df = pd.DataFrame.from_dict(team_rankings["east"], orient='index').sort_values("Wins", ascending=False)

In [ ]:
# Outputting Western conferences team rankings
team_rankings_west_df

In [ ]:
# Outputting Eastern conferences team rankings
team_rankings_east_df

In [ ]:
# Finding advancing teams (more than or equal to 20 wins)
advancing_teams_east = list(team_rankings_east_df[team_rankings_east_df["Wins"] >= 20].index)
advancing_teams_west = list(team_rankings_west_df[team_rankings_west_df["Wins"] >= 20].index)

# Getting all players eligible for the MVP
mvp_candidates = players[(players["TEAM"].isin(advancing_teams_west)) | (players["TEAM"].isin(advancing_teams_east))]

#Initialising fantasy points array
fantasy_points = []

# Calculating fantasy points for each eligible player
for idx, player in mvp_candidates.iterrows():
    fantasy_points.append(3*player["3PM"]+2*(player["FGM"]-player["3PM"])+player["FTM"]+1.2*player["REB"]+1.5*player["AST"]+2*player["STL"]+2*player["BLK"]-player["TO"])

# Adding fantasy points row to MVP candidates matrix
mvp_candidates["Fantasy Points"] = fantasy_points

# Ordering MVP candidates by Fantasy points to determine recipient
mvp_candidates = mvp_candidates.sort_values(by="Fantasy Points", ascending=False)

# Outputting MVP candidates matrix
mvp_candidates

In [9]:
# Outputting MVP
print("Season MVP from wins >= 20 method is " + list(mvp_candidates.head(1)["PLAYER"])[0])

Season MVP from wins >= 20 method is Nikola Jokic
